In [1]:
import urllib.request
from konlpy.tag import Okt
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import json
import pandas as pd

path = '/repo/course/sem21_01/youtube_summarizer/dataset/article_dataset/train.jsonl'

### bfly train
sent_df = pd.read_json(path, lines=True, encoding="utf-8")
print('shape : {}'.format(sent_df.shape))
sent_df.head()

shape : (260697, 5)


,media,id,article_original,abstractive,extractive
0,부산일보,360972161,[지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...,통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 관련 통계...,"[4, 11, 18]"
1,중도일보,356659913,[서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...,서산시 가충순 의원과 이수의 의원이 활발한 의정활동을 펼친 감사의 표시로 한국지역신...,"[1, 3, 4]"
2,무등일보,351718460,[지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...,‘조선대의 새로운 비상을 꿈꾸다’를 슬로건으로 진행되어 단체생활을 통해 협동심과 ...,"[0, 2, 4]"
3,이데일리,335868123,[서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...,서울시가 다음달 4일부터 서울 시내 319개 고등학교 3학년 8만4700명을 대상으...,"[0, 1, 2]"
4,서울신문,351443347,[미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...,미국인 선교사가 우간다에서 의사 행세를 하며 두 아이의 죽음과 관련돼 있다며 지역 ...,"[0, 1, 2]"


In [3]:
# path 추가
sys.path.append('/repo/course/sem21_01/youtube_summarizer/src/word_embedding')

In [4]:
from sent_tokenizer import get_sent_token, get_sent_embedding

In [5]:
article = sent_df['article_original'] 
article.head()

0    [지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작...
1    [서산시의회(의장 임재관) 가충순·이수의 의원이 (사)한국지역신문협회에서 수여하는 ...
2    [지난 2004년 시작해 조선대 학생들의 대표적인 행사로 자리매김한 ‘조선대 국토대...
3    [서울시는 신학기가 시작되는 다음달 4일부터 고등학교 3학년 무상급식을 실시한다고 ...
4    [미국인 선교사가 우간다에서 의사 행세를 하며 의료 시설을 운영한 혐의로 지역 시민...
Name: article_original, dtype: object

In [6]:
article[0][0]

'지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성 이후 가장 많았다.'

In [7]:
sent_token = get_sent_token(article[0][0])
print(sent_token)
sent_vec = get_sent_embedding(sent_token)
print(sent_vec)

['지난해', '고령화', '유례', '드문', '겨울', '한파', '등', '영향', '우리나라', '사망자', '수', '통계', '작성', '이후', '가장', '많', '았', '다']
[ 3.80113870e-02 -6.08295873e-02  9.07433629e-02 -3.06757003e-01
  6.91407844e-02 -4.50094491e-02 -1.37116253e-01 -1.04391657e-01
 -1.56337991e-01 -8.18696320e-02  3.84670869e-02  1.14223756e-01
 -4.78612632e-02 -9.38188583e-02  2.00230986e-01  2.07123339e-01
  9.49049890e-02  2.82414816e-02 -8.62083957e-03  2.13286057e-02
 -1.09993257e-01 -1.06198683e-01 -1.05264194e-01  1.50307357e-01
  5.83401322e-02 -5.83107136e-02  4.08998281e-02  3.59957963e-02
  1.27910137e-01  4.58584391e-02  2.16744855e-01  1.38322175e-01
  1.72423810e-01  5.83438687e-02  6.31604269e-02  7.17057195e-03
  1.80084050e-01 -9.54588354e-02 -2.15125203e-01  1.24519140e-01
 -2.34503914e-02 -2.73202285e-02 -9.79958847e-02 -1.00490659e-01
  2.58664966e-01  7.01965094e-02  8.17812085e-02  1.58336908e-01
  9.22032371e-02  1.28253162e-01 -3.13845836e-02  7.12351734e-03
 -4.47665043e-02 -1.17604055e-01  3.03862430e-

## 환경 셋팅

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

In [11]:
import os
import sys
import numpy as np
from tqdm import tqdm
import time
import pickle

In [13]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [14]:
device

device(type='cuda', index=0)

In [14]:
# path 추가
sys.path.append('/repo/course/sem21_01/youtube_summarizer/src/subtext')


In [15]:
import train
import test
import make_data
import model.subtext_classifier as subtext_classifier

## Load Dataset

In [8]:
window_size = 3

In [17]:
data_path = f'/repo/course/sem21_01/youtube_summarizer/dataset/subtext_dataset/nn_dataset_w{window_size}_fixed.pkl'

with open(data_path, 'rb') as rr:
    dataset = pickle.load(rr)

dataset_fin = [data.permute(0, 2, 1) for data in dataset]

train_x = torch.cat((dataset_fin[0], dataset_fin[1]), dim=0).to(device)
train_y = torch.cat((torch.ones(len(dataset_fin[0])), torch.zeros(len(dataset_fin[1]))), dim=0).to(device)

val_x = torch.cat((dataset_fin[2], dataset_fin[3]), dim=0).to(device)
val_y = torch.cat((torch.ones(len(dataset_fin[2])), torch.zeros(len(dataset_fin[3]))), dim=0).to(device)

test_x = torch.cat((dataset_fin[4], dataset_fin[5]), dim=0).to(device)
test_y = torch.cat((torch.ones(len(dataset_fin[4])), torch.zeros(len(dataset_fin[5]))), dim=0).to(device)

# batch iterator
batch_size = 256

trainset = data_utils.TensorDataset(train_x, train_y)
train_loader = data_utils.DataLoader(trainset, batch_size = batch_size, shuffle = True)

valset = data_utils.TensorDataset(val_x, val_y)
val_loader = data_utils.DataLoader(valset, batch_size = len(valset), shuffle = True)

testset = data_utils.TensorDataset(test_x, test_y)
test_loader = data_utils.DataLoader(testset, batch_size = len(testset), shuffle = True)


In [18]:
class ChunkClassifier(nn.Module):
    
    def __init__(self, x_features):
        super(ChunkClassifier, self).__init__()
        
        # block1
        self.block1 = nn.Sequential(
            nn.Conv1d(in_channels=768, out_channels=128, kernel_size=4),
            nn.BatchNorm1d(128),
            nn.ReLU()
        )
        
        self.block2 = nn.Sequential(
            nn.Linear(128*3, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            
            nn.Linear(16, 1)
        )
        
    def forward(self, x):
        
        # forward block 1
        out = self.block1(x)
        batch_size = out.size()[0]
        
        out = out.view(batch_size, -1)
        out = self.block2(out)
        
        return out.view(-1)

In [20]:
chunk_model = ChunkClassifier(x_features = 100) # NEED TO TALK

chunk_model.to(device)

ChunkClassifier(
  (block1): Sequential(
    (0): Conv1d(768, 128, kernel_size=(4,), stride=(1,))
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block2): Sequential(
    (0): Linear(in_features=384, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=1, bias=True)
  )
)

# 연습장

In [7]:
sent_df.shape

(260697, 5)

## Load Finetuned Word2Vec

In [30]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

'''
Reason behind 3 files being saved : 
https://stackoverflow.com/questions/43588290/creating-word2vec-model-syn1neg-npy-extension
'''

w2v_path ="/repo/course/sem21_01/youtube_summarizer/dataset/pretrained_word2vec/finetuned_wv.kv"

w2v_model = KeyedVectors.load(w2v_path)

w2v_model


In [11]:
len(article[0])

26

In [14]:
article[0][1]

'폐렴과 치매의 일종인 알츠하이머병은 지난해 사망원인 순위 3위와 9위로 전년보다 각각 한 단계, 두 단계 상승하는 등 노인성 질병에 의한 사망률이 급증하는 추세다.'

In [15]:
article[0][1][1]

'렴'

In [18]:
import pickle


# 참고용
pk_path = '/repo/course/sem21_01/youtube_summarizer/dataset/article_tokenized/tokenized_news_data.pkl'

# load pickle file
with open(pk_path, 'rb') as f:
     tokenized_news_data = pickle.load(f)

In [26]:
from konlpy.tag import Mecab
import re

# define mecab tokenizer
mecab = Mecab()

# define stopwords
stopwords = ['을', '를', '이', '가', '은', '는', '의', '에', '와', '으로', '합니다', '입니다']

In [27]:
# define sentence tokenizer
def sent_tokenizer(sentence, stopwords = stopwords):
    '''
    tokenize input sentence when given using stopwords
    '''

    tok_sent = re.sub('[^가-힣a-z]', ' ', sentence) # 영어 소문자와 한글을 제외한 모든 문자를 제거
    tok_sent = mecab.morphs(tok_sent) # 토큰화
    tok_sent = [word for word in tok_sent if not word in stopwords] # 불용어 제거

    return tok_sent

In [28]:
# test on 1st article

for i in range(len(article[0])):
    print(article[0][i])
    print()

지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성 이후 가장 많았다.

폐렴과 치매의 일종인 알츠하이머병은 지난해 사망원인 순위 3위와 9위로 전년보다 각각 한 단계, 두 단계 상승하는 등 노인성 질병에 의한 사망률이 급증하는 추세다.

‘연령표준화 사망률’(표준인구 10만 명당 사망자 수)은 울산·충북·부산 순으로 높게 나타났다.

■작년 사망자 29만 8820명, 역대 최다

24일 통계청이 발표한 '2018년 사망원인통계'를 보면 지난해 총 사망자 수는 전년 대비 4.7%(1만 3286명) 증가한 29만 8820명으로 관련 통계를 작성한 1983년 이후 가장 많았으며, 5년 연속 증가세를 보였다.

통계청은 인구 구조의 고령화와 지난해 1~2월 유례가 드문 한파 등을 그 원인으로 꼽았다.

지난해 조사망률(인구 10만 명당 사망자 수) 역시 582.5명으로 전년보다 4.5%(25.1명) 증가해 5년 연속 늘었다.

특히 80세 이상의 사망자가 전체 사망자의 절반에 가까운 46.3%로, 10년 전보다 14.3%포인트(P)나 증가했다.

■폐렴·알츠하이머병 사망률 순위 ‘껑충’

사망원인별로 보면 지난해 암(악성신생물)에 의한 사망률(이하 인구 10만 명당 사망자 수)은 154.3명으로 전년보다 0.2% 증가했다.

1983년 관련 통계를 집계한 이래 줄곧 암이 사망원인 1위로 집계됐다.

특히 폐렴(4위→3위)과 치매의 일종인 알츠하이머병(11위→9위)에 의한 사망률 순위 상승이 두드러졌다.

폐렴 사망률은 2004년 10위에서 꾸준히 순위가 상승하고 있고, 알츠하이머병 사망률 역시 통계 작성 이래 10대 사인에 처음 포함됐다.

지난해 알츠하이머병에 의한 사망률은 12.0명으로 전년(9.8명) 대비 22.5% 증가했다.

알츠하이머병 사망률은 10년 전(3.8명)과 비교하면 무려 214.2% 증가했다.

폐렴 사망률은 45.4명으로 전년(37.8명) 대비 20.0% 증가했다.

알코올 관련 사망률은 9.6명으로 전년보다 

In [33]:
w2v_model.get_vector('좋아해')

array([ 0.3782275 ,  0.6562686 , -0.4142072 , -0.38342747,  0.22446448,
        0.12953684,  0.6192602 , -0.47810614,  0.42953876, -0.35954252,
       -0.3518827 ,  0.1778559 ,  0.50612223, -0.38547012,  0.33385023,
        0.39495888,  0.63455206, -0.09762726, -0.0029143 ,  0.4795328 ,
        0.4826215 ,  0.41814062,  0.12996835,  0.5566212 , -0.2807082 ,
        0.61919737,  0.22817875, -0.22229761, -0.56404567, -0.30374673,
        0.04125731,  0.27241135, -0.19335729,  0.3081824 , -0.129556  ,
       -0.00858455, -0.93879247,  0.12960172,  0.10804071,  0.01190971,
       -0.06295747, -0.01805767,  0.3526485 , -0.16974664, -0.06867648,
       -0.19264841, -0.02780545, -0.15973386, -0.5359207 , -0.5074347 ,
        0.38011527, -0.07387166, -0.44733685, -0.50342417, -0.25839671,
        0.49253833,  0.04603034,  0.2572975 ,  0.0349206 , -0.07181429,
       -0.25659448, -0.05567137,  0.2649214 ,  0.3430381 ,  0.38359126,
        0.41281524,  0.31742987,  0.12440167,  0.23134698, -0.38

In [42]:
vec1 = [1,2,3]
vec2 = [1,2,3]
vec3 = [1,2,3]
vec4 = [1,2,3]

vec = []


In [44]:
vec.append(vec1)
vec.append(vec2)
vec.append(vec3)
vec.append(vec4)

vec

[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [45]:
import numpy as np
np.mean(vec, axis =0)

array([1., 2., 3.])

In [47]:
# test on 1st article

for i in range(len(article[0])):
    sen = article[0][i]
    print(sen)
    tokens = sent_tokenizer(sen)
    print(tokens)
    w2v_list = []
    for token in tokens:
        tmp_vec = w2v_model.get_vector(token)
        w2v_list.append(tmp_vec)
        print(len(w2v_list))

    sent_vec = np.mean(w2v_list, axis = 0)

    print(f'sent_vec : \n {sent_vec}')


지난해 고령화와 유례가 드문 겨울 한파 등 영향으로 우리나라 사망자 수가 통계 작성 이후 가장 많았다.
['지난해', '고령화', '유례', '드문', '겨울', '한파', '등', '영향', '우리나라', '사망자', '수', '통계', '작성', '이후', '가장', '많', '았', '다']
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
sent_vec : 
 [-1.06015697e-01  1.09804034e-01  1.85841665e-01  2.54072398e-01
 -3.15754041e-02  1.41379774e-01  1.80125222e-01 -8.54291320e-02
  2.60001093e-01  1.86146125e-02 -1.49292290e-01  3.13859820e-01
  1.24609314e-01 -1.72457621e-01 -1.95455730e-01  1.03890747e-01
 -4.31853384e-02  7.68965036e-02 -1.18152775e-01 -2.21658461e-02
  3.38335633e-01  1.73869491e-01 -2.11152852e-01  1.09010763e-01
 -1.48767665e-01  2.08321497e-01 -7.84329996e-02 -3.10370713e-01
 -3.28147970e-03 -4.29815389e-02 -1.25268176e-01  7.50445575e-02
  2.31080800e-01  1.35232210e-01  9.61280763e-02  1.40447877e-02
 -3.63451838e-01  2.25478649e-01 -1.03481188e-01 -3.73585671e-02
  2.18511745e-01 -2.25801498e-01  3.31162572e-01 -8.76678899e-02
  2.08866179e-01  1.64340273e-01  1.30770758e-01  1.47349417